In [1]:
import os
import numpy as np
import pandas as pd
from sigfig import round as sigfig_round
from collections import defaultdict
from scipy import stats


def get_rounded_metrics(
    predictions_dir,
    task_type,
    task_name,
    target_label,
    split_type,
    model_name_short,
    hyperparameters_str,
):
    metrics_flatten = defaultdict(list)
    metrics_average = defaultdict(list)
    metrics_selfies = defaultdict(list)

    for num_gr_sf in NUM_GROUP_SELFIES_:
        num_group_selfies = str(num_gr_sf)
        preds_dir = os.path.join(
            predictions_dir,
            task_type,
            task_name,
            target_label,
            split_type,
            num_group_selfies,
            model_name_short,
            hyperparameters_str,
        )
        for subdir, dirs, files in os.walk(preds_dir):
            if len(files) > 0:
                for file in files:
                    if file == "y_true_flatten.npy":
                        labels = np.load(os.path.join(subdir, file))
                    if file == "y_pred_flatten.npy":
                        preds = np.load(os.path.join(subdir, file))
                    if file == "y_true_average.npy":
                        y_true_average = np.load(os.path.join(subdir, file))
                    if file == "y_pred_average.npy":
                        y_pred_average = np.load(os.path.join(subdir, file))
                    if file == "y_true_selfies.npy":
                        y_true_sf = np.load(os.path.join(subdir, file))
                    if file == "y_pred_selfies.npy":
                        y_pred_sf = np.load(os.path.join(subdir, file))

                num_group_selfies = int(subdir.split("/")[5])

                metrics = {}
                metrics["pearson"] = stats.pearsonr(labels, preds).statistic.item()
                metrics_flatten[num_group_selfies].append(metrics)

                metrics = {}
                metrics["pearson"] = stats.pearsonr(
                    y_true_average, y_pred_average
                ).statistic.item()
                metrics_average[num_group_selfies].append(metrics)

                metrics = {}
                metrics["pearson"] = stats.pearsonr(
                    y_true_sf, y_pred_sf
                ).statistic.item()
                metrics_selfies[num_group_selfies].append(metrics)
    print("\nflatten metrics")
    for gs_num in NUM_GROUP_SELFIES_:
        rmse_current = np.array([m["pearson"] for m in metrics_flatten[gs_num]])
        print(
            f"{str(gs_num).rjust(2)} Group SELFIES, Pearson coef.:\t",
            sigfig_round(
                np.mean(rmse_current), uncertainty=np.std(rmse_current), style="PDG"
            ),
            f"\t({rmse_current.size} runs)",
        )

    print("\nensemble metrics")
    for gs_num in NUM_GROUP_SELFIES_:
        rmse_current = np.array([m["pearson"] for m in metrics_average[gs_num]])
        print(
            f"{str(gs_num).rjust(2)} Group SELFIES, Pearson coef.:\t",
            sigfig_round(
                np.mean(rmse_current), uncertainty=np.std(rmse_current), style="PDG"
            ),
            f"\t({rmse_current.size} runs)",
        )

    print("\nSELFIES metrics")
    for gs_num in NUM_GROUP_SELFIES_:
        rmse_current = np.array([m["pearson"] for m in metrics_selfies[gs_num]])
        print(
            f"{str(gs_num).rjust(2)} Group SELFIES, Pearson coef.:\t",
            sigfig_round(
                np.mean(rmse_current), uncertainty=np.std(rmse_current), style="PDG"
            ),
            f"\t({rmse_current.size} runs)",
        )


NUM_GROUP_SELFIES_ = [1]

#### Sol

In [2]:
predictions_dir = "predictions"
task_type = "regression"
task_name = "Sol"
target_label = "activity"
split_type = "original"

MODEL_NAME = "korolewadim/gselformer-large"

BATCH_SIZE = 32
NUM_EPOCHS = 50
WARMUP_RATIO = 0.0
LEARNING_RATE = 2e-5
hyperparameters_str = f"batch_size_{BATCH_SIZE}__num_epochs_{NUM_EPOCHS}__warmup_ratio_{WARMUP_RATIO}__lr_{LEARNING_RATE}"

get_rounded_metrics(
    predictions_dir,
    task_type,
    task_name,
    target_label,
    split_type,
    MODEL_NAME,
    hyperparameters_str,
)


flatten metrics
 1 Group SELFIES, Pearson coef.:	 0.618 ± 0.005 	(5 runs)

ensemble metrics
 1 Group SELFIES, Pearson coef.:	 0.636 ± 0.005 	(5 runs)

SELFIES metrics
 1 Group SELFIES, Pearson coef.:	 0.630 ± 0.007 	(5 runs)
